In [43]:
import aoc

In [ ]:
grid = aoc.read("day3.txt", to_grid=True)
rows = len(grid)
cols = len(grid[0])
rows, cols

In [4]:
def check_valid_number(row, col, grid):
    for r_add in -1, 0, 1:
        if not aoc.is_outside_bounds((row + r_add, col), (rows, cols)):
            val = grid[row + r_add][col]
            if val != "." and not val.isdigit():
                return True

    return False

In [ ]:
valid_numbers = []
for r in range(rows):
    number = ""
    checking_number = False
    valid_number = False

    for c in range(cols):
        value = grid[r][c]
        if value.isdigit():
            number += value
            if not checking_number:
                valid_number = (
                    check_valid_number(r, c - 1, grid)
                    or check_valid_number(r, c, grid)
                    or check_valid_number(r, c + 1, grid)
                )
            elif not valid_number:
                valid_number = check_valid_number(r, c + 1, grid)
            checking_number = True
        else:
            if checking_number and valid_number:
                valid_numbers.append(int(number))
            checking_number = False
            valid_number = False
            number = ""
        if c == cols - 1:
            if checking_number and valid_number:
                valid_numbers.append(int(number))

sum(valid_numbers)

# Part 2

In [ ]:
def find_surrounding_numbers(r, c, grid):
    surrounding_directions = [
        (-1, -1),
        (-1, 0),
        (-1, 1),
        (0, -1),
        (0, 1),
        (1, -1),
        (1, 0),
        (1, 1),
    ]
    are_digits = [
        grid[r + dir_[0]][c + dir_[1]].isdigit() for dir_ in surrounding_directions
    ]

    to_be_expanded = []
    if are_digits[0]:
        if not are_digits[1]:
            to_be_expanded.append((((r - 1, c - 1), -1)))
        elif not are_digits[2]:
            to_be_expanded.append((((r - 1, c), -1)))
        else:
            to_be_expanded.append((((r - 1, c - 1), 0)))
    elif are_digits[1]:
        to_be_expanded.append(((r - 1, c), 1))
    if are_digits[2] and not are_digits[1]:
        to_be_expanded.append(((r - 1, c + 1), 1))

    if are_digits[3]:
        to_be_expanded.append(((r, c - 1), -1))
    if are_digits[4]:
        to_be_expanded.append(((r, c + 1), 1))

    if are_digits[5]:
        if not are_digits[6]:
            to_be_expanded.append((((r + 1, c - 1), -1)))
        elif not are_digits[7]:
            to_be_expanded.append((((r + 1, c), -1)))
        else:
            to_be_expanded.append((((r + 1, c - 1), 0)))
    elif are_digits[6]:
        to_be_expanded.append(((r + 1, c), 1))
    if are_digits[7] and not are_digits[6]:
        to_be_expanded.append(((r + 1, c + 1), 1))
    return [expand_numbers(*pos, direction, grid) for pos, direction in to_be_expanded]


def expand_numbers(row, col, direction, grid=grid) -> str:
    if direction == 0:
        backward = expand_numbers(row, col, -1, grid)
        forward = expand_numbers(row, col + 1, 1, grid)
        return backward + forward

    number = ""
    c = col
    while True:
        if aoc.is_outside_bounds((row, c), (rows, cols)):
            break
        value = grid[row][c]
        if not value.isdigit():
            break
        if direction == -1:
            number = value + number
        if direction == 1:
            number = number + value
        c += direction

    # We must return a string, and not a number; otherwise the direction == 0
    # would add '8' and '09'  as 89 (since the int of 09 would be 9)
    return number


def find_gear_ratio(star):
    surrounding_numbers = find_surrounding_numbers(*star, grid)
    if len(surrounding_numbers) == 2:
        return int(surrounding_numbers[0]) * int(surrounding_numbers[1])
    return 0


all_stars = [(r, c) for r in range(rows) for c in range(cols) if grid[r][c] == "*"]
sum(find_gear_ratio(star) for star in all_stars)